# Constants

## Transport

Ratio par jour à partir des modes de transport + distance des cadres

- Moyenne distance travail cadre: 19km [Observatoire des territoires](https://www.observatoire-des-territoires.gouv.fr/kiosque/2019-mobilite-10-les-cadres-parcourent-chaque-annee-pres-de-3000-km-de-plus-que-les)
- Ratio Déplacements cadres et professions intellectuelles, toutes activités dans un rayon de 80km: [INSEE](https://www.statistiques.developpement-durable.gouv.fr/resultats-detailles-de-lenquete-mobilite-des-personnes-de-2019?rubrique=60&dossier=1345)
    1. Marche à pied: 1.3818
    2. Vélo: 1.44675
    3. Transport en commun: 17.77558
    4. Voiture: 76.32407
    5. Deux roues motorisé: 2.91722
    6. Autre: 0.15459

- Emissions transports:
    - Voiture - Motorisation moyenne - 2018: 
        - 0.218 kg CO2-eq [ADEME](https://bilans-ges.ademe.fr/fr/basecarbone/donnees-consulter/liste-element/categorie/151)
    - Vélo
        - Moyenne fabrication [Trek](https://view.publitas.com/trek-bicycle/trek-bicycle-2021-sustainability-report/page/5): 174kg CO2
        - 50 000km (voir notes)
    - Métro, tramway, trolleybus - 2018 - Agglomération de 100 000 à 250 000 habitants
        - 0.00503 kgCO2e/passager.km [ADEME](https://bilans-ges.ademe.fr/fr/accueil/documentation-gene/index/page/Ferroviaire2)
    - Moto > 250 cm3 - Mixte - 2018 
        - 0.191 kgCO2e/km [ADEME](https://bilans-ges.ademe.fr/fr/accueil/documentation-gene/index/page/Routier2)


In [10]:
CAR_CO2_KM = 0.218
BIKE_CO2_KM = 174 / 50000  # 0,00348
PUBLIC_TRANSPORT_CO2_KM = 0.00503
MOTORBIKE_CO2_KM = 0.191

FOOT_PERCENTAGE = 1.3818
BIKE_PERCENTAGE = 1.44675
PUBLIC_TRANSPORT_PERCENTAGE = 17.77558
CAR_PERCENTAGE = 76.32407 + 0.15459  # other categories (tractors...)
MOTORBIKE_PERCENTAGE = 2.91722

AVG_KM_DAY = 19 * 2

mixed_emissions_km = (
                             FOOT_PERCENTAGE * 0
                             + BIKE_PERCENTAGE * BIKE_CO2_KM
                             + PUBLIC_TRANSPORT_PERCENTAGE * PUBLIC_TRANSPORT_CO2_KM
                             + CAR_PERCENTAGE * CAR_CO2_KM
                             + MOTORBIKE_PERCENTAGE * MOTORBIKE_CO2_KM
                     ) / 100

transport_co2_person = mixed_emissions_km * AVG_KM_DAY
print(transport_co2_person)

6.583113447812


## Buildings

- Surface par salarié: 18 m2/poste avec un taux de bureaux de 70% [Legifrance](https://www.legifrance.gouv.fr/jorf/id/JORFTEXT000042994780)
- ACV bureaux: 3,9 t eqCO2/m2 [Observatoire de l'immobilier durable](https://resources.taloen.fr/resources/documents/7765_191210_poids_carbone_ACV_vdef.pdf)
    - 2,3t travaux
    - 1,6t exploitation
    - 50 ans d'utilisation

In [11]:
OFFICE_EMISSIONS_SQR_METER_DAY = 3900 / (50 * 365)  # 0,21
# 18m2/office are 70% of the building
office_co2_person = ((18 * 100) / 70) * OFFICE_EMISSIONS_SQR_METER_DAY

print(office_co2_person)
manday_co2 = office_co2_person + transport_co2_person

5.495107632093934


## Servers

- Electricité - 2020 - mix moyen - consommation: 0.0599 kgCO2e/kWh [ADEME](https://bilans-ges.ademe.fr/fr/accueil/documentation-gene/index/page/Electricite_reglementaire)
- Serveur [Boavitza](https://github.com/Boavizta/environmental-footprint-data)
    - Fabrication: 1613.25
    - Life expectency: 3.89
    - Conso RUN = 1100 (Orange) 
    - Conso IDLE = 234 (Orange)

In [12]:
ELECTRICITY_MIX = 0.0599

SERVER_POWER_IDLE = 234
SERVER_POWER_RUN = 1100
SERVER_LIFE = 3.89
SERVER_FABRICATION_CO2 = 1613.25
SERVER_USAGE  = 0.7
PUE = 1.5

amortization_day = SERVER_FABRICATION_CO2 / (SERVER_LIFE * 365)

wh = (SERVER_POWER_RUN - SERVER_POWER_IDLE) * SERVER_USAGE + SERVER_POWER_IDLE
wh_pue = wh * PUE
kwh_day = (wh_pue * 24) / 1000
server_day = kwh_day * ELECTRICITY_MIX + amortization_day
print(server_day)

2.9480188516448917


## Disks

- 250 kg CO2-eq pour un disque 4TO
- 4 ans (Orange)
- 0,89 W/h HDD, 1,52 W/h SSD pour 1 TO (Orange)

In [13]:
HDD_TB_CONS = 0.89
SSD_TB_CONS = 1.52

DISK_SIZE = 4
DISK_LIFE = 4
DISK_FABRICATION_CO2 = 250

wh_to = SSD_TB_CONS
wh_disk = DISK_SIZE * wh_to
wh_pue = wh_disk * PUE
kwh_day = (wh_pue * 24) / 1000
disk_day = kwh_day * ELECTRICITY_MIX

print(disk_day)

0.013110912000000002


## Network

- 1,5g / GO, (Orange)

In [14]:
gb_co2 = 0.0015

## Users

- Smartphone: 
    - Par jour: 
        - 3,12h [Article](https://ieeexplore.ieee.org/abstract/document/6360448)
        - 2,79 [Article](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7960745/)

- Ordinateur portable :
    - [ADEME](https://bilans-ges.ademe.fr/documentation/UPLOAD_DOC_FR/index.htm?ordinateurs_et_equuipements_pe.htm):
        - 169 kg CO2-eq 
        - 5,6 ans [EU](https://www.eup-network.de/fileadmin/user_upload/Produktgruppen/Lots/Final_Documents/EuP_Lot3_PC_FinalReport.pdf)
    - [Boavitza](https://github.com/Boavizta/environmental-footprint-data)
        - 307.37 kg
        - 4.34 ans

Ratio 50/50 accès smartphone/laptop

In [15]:
SMARTPHONE_CO2 = 88.75
SMARTPHONE_LIFE = 2
SMARTPHONE_DAILY_USE = 3.12

LAPTOP_CO2 = 307.37
LAPTOP_LIFE = 4.34
PC_DAILY_USE = 7


smartphone_day_co2 = SMARTPHONE_CO2 / (SMARTPHONE_LIFE * 365)
smartphone_hour_co2 = smartphone_day_co2 / SMARTPHONE_DAILY_USE

laptop_day_co2 = LAPTOP_CO2 / (LAPTOP_LIFE * 365)
laptop_hour_co2 = laptop_day_co2 / PC_DAILY_USE

user_hour_co2 = laptop_hour_co2 * 0.5 + smartphone_hour_co2 * 0.5

print(user_hour_co2)

0.0333428327821832


In [16]:
print('Transport/person/day = ', transport_co2_person)
print('Office/person/day = ', office_co2_person)
print('Man-day co2 = ', manday_co2)

print('Server day = ', server_day)
print('CO2/Gb = ', gb_co2)
print('Disk day = ', disk_day)

print('User hour co2 = ', user_hour_co2)

Transport/person/day =  6.583113447812
Office/person/day =  5.495107632093934
Man-day co2 =  12.078221079905934
Server day =  2.9480188516448917
CO2/Gb =  0.0015
Disk day =  0.013110912000000002
User hour co2 =  0.0333428327821832


# Model

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt
from IPython import display
import ipywidgets as widgets

class Model:
    """
    Software lifecycle carbon cost model class
    """
    def __init__(self):
        #Inputs
        self.inputs = {
            "disks": ModelInput(10, 3000, 'Number of disks: '),
            "servers" : ModelInput(6, 300, 'Number of servers : '),
            "data_transferred" : ModelInput(1000000, 30000000, 'Data transferred: '),
            "user_hours" : ModelInput(150000, 300000, 'Users hours:'),
            "spec" : ModelInput(200, 300, 'Specifications man-days:'),
            "dev" : ModelInput(2000, 3000, 'Dev man-days:'),
            "design" : ModelInput(130, 3000, 'Design man-days:'),
            "test" : ModelInput(200, 3000, 'Test man-days:'),
            "dev_servers" : ModelInput(4, 30, 'Number of dev servers:'),
            "support" : ModelInput(50, 300, 'Support man-days:'),
            "maintenance" : ModelInput(700, 3000, 'Maintenance man-days:'),
            "run_length" : ModelInput(365, 3000, 'Run phase duration:'),
            "build_length" : ModelInput(365, 3000, 'Build phase duration:'),
        }
        self.project_length = None

        # Outputs
        self.network_co2 = None
        self.management_co2 = None
        self.support_co2 = None
        self.servers_co2 = None
        self.implementation_co2 = None
        self.spec_co2 = None
        self.test_co2 = None
        self.design_co2 = None
        self.dev_co2 = None
        self.run_co2 = None
        self.build_co2 = None
        self.infra_dev_co2 = None
        self.maintenance_co2 = None
        self.storage_co2 = None
        self.infra_co2 = None
        self.user_co2 = None

        self.compute()

    def compute(self):
        """
        Compute the phases carbon cost retrieving the inputs from the sliders
        """
        self.project_length = self.inputs["run_length"].get_value() - self.inputs["build_length"].get_value()
        self.compute_build()
        self.compute_run()

        
    def compute_build(self):
        """
        Compute the build phase outputs
        """

        ## Requirement analysis and specifications
        self.spec_co2 = self.inputs["spec"].get_value() * MAN_DAY_CO2

        ## Implementation
        self.infra_dev_co2 = self.inputs["dev_servers"].get_value() * SERVER_DAY * self.inputs["build_length"].get_value()
        self.dev_co2 = self.inputs["dev"].get_value() * MAN_DAY_CO2
        self.design_co2 = self.inputs["design"].get_value() * MAN_DAY_CO2
        self.test_co2 = self.inputs["test"].get_value() * MAN_DAY_CO2

        self.implementation_co2 = self.inputs["dev"].get_value() * MAN_DAY_CO2 + self.infra_dev_co2

        # Management
        self.management_co2 = self.inputs["build_length"].get_value() * MAN_DAY_CO2

        # Global
        self.build_co2 = self.spec_co2 + self.implementation_co2 + self.management_co2

    def compute_run(self):
        """
        Compute the run phase outputs
        """

        # Support
        self.support_co2 = self.inputs["support"].get_value() * MAN_DAY_CO2

        # Maintenance
        self.maintenance_co2 = self.inputs["maintenance"].get_value() * MAN_DAY_CO2

        # Infra
        self.network_co2 = self.inputs["data_transferred"].get_value() * CO2_GB
        self.servers_co2 = self.inputs["servers"].get_value() * self.inputs["run_length"].get_value() * SERVER_DAY
        self.storage_co2 = self.inputs["disks"].get_value() * self.inputs["run_length"].get_value() * DISK_DAY

        self.infra_co2 = self.network_co2 + self.servers_co2 + self.storage_co2

        # User
        self.user_co2 = self.inputs["user_hours"].get_value() * USER_HOUR_CO2

        # Global
        self.run_co2 = self.support_co2 + self.maintenance_co2 + self.infra_co2 + self.user_co2


class ModelInput:
    """
    Inputs from the model, gives sliders and corresponding values
    """
    def __init__(self, value, max_value, desc):
        style = {'description_width': 'initial'}
        self.slider = widgets.IntSlider(min=0, max=max_value, value= value, description = desc, layout=widgets.Layout(width='400px'), style = style, readout=False)
        d = widgets.IntText(continuous_update=True, layout=widgets.Layout(width='80px'))
        widgets.link((self.slider, 'value'), (d, 'value'))
        self.widget = widgets.HBox([self.slider, d])

    def get_value(self):
        return self.slider.value

# Output

In [18]:
def draw_model(model: Model):
    """
    Pie chart model representation
    :param model: Model to pie
    """
    fig1, ax1 = plt.subplots()
    ax1.pie([model.build_co2, model.run_co2], labels=["Build", "Run"])
    ax1.set_title("Global")

    fig, ((ax2, ax3), (ax4, ax5)) = plt.subplots(2, 2, constrained_layout=True)
    ax2.pie(
        [model.spec_co2, model.implementation_co2, model.management_co2],
        labels=["Spec", "Implementation", "Management"],
    )
    ax2.set_title("Build")
    ax3.pie(
        [model.dev_co2, model.design_co2, model.test_co2, model.infra_dev_co2],
        labels=["Dev", "Design", "Test", "Infra"],
    )
    ax3.set_title("Implementation")

    ax4.pie(
        [model.support_co2, model.maintenance_co2, model.infra_co2, model.user_co2],
        labels=["Support", "Maintenance", "Infra", "User"],
    )
    ax4.set_title("Run")
    ax5.pie(
        [model.network_co2, model.storage_co2, model.servers_co2],
        labels=["Network", "Storage", "Compute"],
    )
    ax5.set_title("Infra")

    datas = [
        model.network_co2,
        model.management_co2,
        model.support_co2,
        model.servers_co2,
        model.spec_co2,
        model.test_co2,
        model.design_co2,
        model.dev_co2,
        model.infra_dev_co2,
        model.maintenance_co2,
        model.storage_co2,
        model.user_co2,
    ]
    labels = [
        "network_co2",
        "management_co2",
        "support_co2",
        "servers_co2",
        "spec_co2",
        "test_co2",
        "design_co2",
        "dev_co2",
        "infra_dev_co2",
        "maintenance_co2",
        "storage_co2",
        "user_co2",
    ]

    fig3, ax6 = plt.subplots()
    ax6.pie(datas, labels=labels)
    ax6.set_title("Global")

    plt.show()


model = Model()

# Button refresh
button_refresh = widgets.Button(description="Refresh")


def on_button_clicked(b):
    with out:
        display.clear_output(True)
        model.compute()
        draw_model(model)


button_refresh.on_click(on_button_clicked)

# Sliders
vbox_build = widgets.VBox(
    [
        widgets.Text(value="Build: ", disabled=True),
        model.inputs["spec"].widget,
        model.inputs["dev"].widget,
        model.inputs["design"].widget,
        model.inputs["test"].widget,
        model.inputs["dev_servers"].widget,
        model.inputs["build_length"].widget,
    ]
)

vbox_run = widgets.VBox(
    [
        widgets.Text(value="Run: ", disabled=True),
        model.inputs["disks"].widget,
        model.inputs["servers"].widget,
        model.inputs["data_transferred"].widget,
        model.inputs["user_hours"].widget,
        model.inputs["support"].widget,
        model.inputs["maintenance"].widget,
        model.inputs["run_length"].widget,
    ]
)

vbox_sliders = widgets.VBox([vbox_build, vbox_run, button_refresh])

# Graph
out = widgets.Output()
with out:
    draw_model(model)


# Global
widgets.HBox([vbox_sliders, out])


NameError: name 'MAN_DAY_CO2' is not defined